In [1]:
import warnings

warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments

df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.


### 1
How many different cases are there, by department?

### 2
Does the percentage of cases that are late vary by department?

### 3
On average, how late are the late cases by department?

### 4
What is the service type that is the most late? Just for Parks & Rec?

### 5
For the DSD/Code Enforcement department, what are the most common service request types? 
Look at other departments too.

### 6
Does whether or not its a weekend matter for when a case is opened/closed?

### 7
On average, how many cases are opened a day for the Customer Service department?

### 8
Does the number of service requests for the solid waste department vary by day of the week?